In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt, atan2
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.edges.layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df  = pl.DataFrame(_lu_)

In [ ]:
class SingleNodeFocus(object):
    #
    #
    #
    def __init__(self, rt_self, df, relationships, node_focus=None, count_by=None, count_by_set=False, color_by=None, 
                 chord_diagram_max_r = 256, chord_diagram_min_r = 224,
                 txt_h=12, w=768, h=768):
        self.rt_self              = rt_self
        self.df                   = df    
        self.relationships        = relationships
        self.count_by             = count_by
        self.count_by_set         = count_by_set
        self.color_by             = color_by
        self.chord_diagram_max_r  = chord_diagram_max_r
        self.chord_diagram_min_r  = chord_diagram_min_r
        self.txt_h                = txt_h
        self.w                    = w
        self.h                    = h
        self.last_render          = None
        self.g                    = self.rt_self.createNetworkXGraph(self.df, self.relationships)
        self.node_focus           = node_focus
        if self.node_focus is None: self.node_focus = list(self.g.nodes())[0]
        self.cx, self.cy          = self.w/2, self.h/2
    
    #
    #
    #
    def _repr_svg_(self):
        if self.last_render is None: self.renderSVG()
        return self.last_render

    #
    #
    #
    def __partitionNeighbors__(self, g, node):
        _partitions_ = {1:[], 2:[], 3:[], 10:[], 100:[], 101:[]}
        for _nbor_ in g.neighbors(node):
            if   g.degree[_nbor_] == 1:   _partitions_[1]  .append(_nbor_)
            elif g.degree[_nbor_] == 2:   _partitions_[2]  .append(_nbor_)
            elif g.degree[_nbor_] == 3:   _partitions_[3]  .append(_nbor_)
            elif g.degree[_nbor_] <= 10:  _partitions_[10] .append(_nbor_)
            elif g.degree[_nbor_] <= 100: _partitions_[100].append(_nbor_)
            else:                         _partitions_[101].append(_nbor_)
        return _partitions_

    def __renderImmediateNeighbors__(self):
        svg = []
        _fm_, _to_ = self.relationships[0][0], self.relationships[0][1]
        _set_         = set([self.node_focus])
        _df_nbors_    = self.df.filter(pl.col(_fm_).is_in(_set_) | pl.col(_to_).is_in(_set_))
        _set_local_   = (set(_df_nbors_['fm']) | set(_df_nbors_['to']))
        _set_nbors_   = _set_local_ - _set_
        _df_subgraph_ = self.df.filter(  pl.col(_fm_).is_in(_set_local_) & pl.col(_to_).is_in(_set_local_))
        _df_ee_       = self.df.filter(~(pl.col(_fm_).is_in(_set_local_) | pl.col(_to_).is_in(_set_local_)))
        g_ee          = self.rt_self.createNetworkXGraph(_df_ee_, self.relationships)
        communities   = nx.community.louvain_communities(g_ee)
        clusters      = {}
        if type(self.node_focus) is int:
            community_base = max(set(self.df[_fm_]) | set(self.df[_to_])) + 1000
            for i in range(len(communities)):
                clusters[community_base + i] = list(communities[i])
        elif type(self.node_focus) is str:
            for i in range(len(communities)):
                clusters[f'__cluster_{i}__'] = list(communities[i])
        else:
            raise Exception(f'Unknown type for node_focus: {type(self.node_focus)}')

        _df_collapse_ = rt.collapseDataFrameGraphByClusters(self.df, self.relationships, clusters)
        g_collapse    = self.rt_self.createNetworkXGraph(_df_collapse_, [('__fm__','__to__')])
        pos           = nx.spring_layout(g_collapse)

        r_inner, r_outer = self.w/5.0, self.w/2 - self.chord_diagram_max_r/2.0 - 10.0
        for _node_ in pos:
            if _node_ == self.node_focus: continue
            wx, wy  = pos[_node_]
            dx, dy  = wx - pos[self.node_focus][0], wy - pos[self.node_focus][1]
            if dx == 0.0 and dy == 0.0: dx = 1.0
            _angle_ = atan2(dy, dx)
            if _node_ in clusters.keys():
                self.pos[_node_] = (self.cx + r_outer * cos(_angle_), self.cy + r_outer * sin(_angle_))
            else:
                self.pos[_node_] = (self.cx + r_inner * cos(_angle_), self.cy + r_inner * sin(_angle_))
        
        for _node_ in pos:
            for _nbor_ in g_collapse.neighbors(_node_):
                sx, sy = self.pos[_node_]
                ex, ey = self.pos[_nbor_]
                svg.append(f'<line x1="{sx}" y1="{sy}" x2="{ex}" y2="{ey}" stroke="gray" stroke-width="0.2" />')
        
        for _node_ in pos:
            if _node_ == self.node_focus: continue
            sx, sy = self.pos[_node_]
            svg.append(f'<circle cx="{sx}" cy="{sy}" r="5" stroke="black" stroke-width="2" fill="{self.rt_self.co_mgr.getTVColor('data','default')}" />')
            svg.append(self.rt_self.svgText(f"{_node_}", sx, sy + self.txt_h + 4, 'black', anchor='middle'))

        return ''.join(svg)

    #
    #
    #
    def renderSVG(self):
        self.pos = {self.node_focus: (self.cx, self.cy)}
        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}">']
        svg.append(f'<rect width="{self.w}" height="{self.h}" x="0" y="0" fill="{self.rt_self.co_mgr.getTVColor('background','default')}" />')
        svg.append(self.__renderImmediateNeighbors__())
        svg.append(f'<circle cx="{self.cx}" cy="{self.cy}" r="5" stroke="black" stroke-width="2" fill="{self.rt_self.co_mgr.getTVColor('data','default')}" />')
        svg.append(self.rt_self.svgText(f"{self.node_focus}", self.cx, self.cy + self.txt_h + 4, 'black', anchor='middle'))
        self.last_render = ''.join(svg) + '</svg>'

#
nodes_as_list, _table_ = list(set(my_df['fm']) | set(my_df['to'])), []
for i in range(4):
    _node_ = random.choice(nodes_as_list)
    _snf_  = SingleNodeFocus(rt, my_df, [('fm','to')], node_focus=_node_, w=768, h=768)
    _table_.append(_snf_)
rt.table(_table_, per_row=2, spacer=10)

In [ ]:
#
# Filter Operator To Remove Any Edges That Contain Either Node in Set
#
df = pl.DataFrame({'fm':'a b c d e f g'.split(),
                   'to':'b c d e f g a'.split()})
_set_ = set(['a','b','c','d'])
df.filter(~(pl.col('fm').is_in(_set_) | pl.col('to').is_in(_set_)))

In [ ]:
g = rt.createNetworkXGraph(my_df, [('fm','to')])
_list_ = nx.community.louvain_communities(g)
for i in range(len(_list_)):
    print(_list_[i])

In [ ]:
_gen_ = nx.community.louvain_partitions(g)
for _lists_ in _gen_:
    for i in range(len(_lists_)):
        print(_lists_[i])
    print('---')

In [ ]:
_dv_ = nx.community.label_propagation_communities(g)
for k in _dv_:
    print(k)